In [18]:
import torch
emb = torch.load("datasets/AIC20/embeddings/S01/gt.txt/node/c001/138.pt")

In [26]:
for e in emb:
    print(e[1:].size())

torch.Size([768])
torch.Size([768])


In [10]:
import pandas as pd
df = pd.read_csv("datasets/AIC20/annotations/S01/c001/gt.txt")
df[df.frame == 138]
for _, row in df[df.frame == 138].iterrows():
    print(row)

frame        138
id             8
xmin         396
ymin         372
width        379
height       156
lost           1
occluded      -1
generated     -1
label         -1
xmax         775
ymax         528
Name: 92, dtype: int64
frame         138
id              9
xmin          568
ymin          492
width         702
height        273
lost            1
occluded       -1
generated      -1
label          -1
xmax         1270
ymax          765
Name: 93, dtype: int64


In [6]:
df.index

Index([  56,   57,   58,   59,   60,   61,   62,   63,   64,   65,
       ...
       1950, 1950, 1951, 1951, 1952, 1952, 1953, 1953, 1954, 1955],
      dtype='int64', name='frame', length=2946)

In [6]:
import torch
from swin_transformer import swin_tiny_patch4_window7_224,swin_small_patch4_window7_224,swin_base_patch4_window7_224

model_path = 'swin_tiny.pth'
semantic_weight = 1.0

swin = swin_tiny_patch4_window7_224(convert_weights=False, semantic_weight=semantic_weight)
swin.init_weights(model_path)
feat_dim = swin.num_features[-1]
print(swin)

unloaded parameters: <All keys matched successfully>
SwinTransformer(
  (patch_embed): PatchEmbed(
    (adap_padding): AdaptivePadding()
    (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (drop_after_pos): Dropout(p=0.0, inplace=False)
  (stages): ModuleList(
    (0): SwinBlockSequence(
      (blocks): ModuleList(
        (0-1): 2 x SwinBlock(
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): ShiftWindowMSA(
            (w_msa): WindowMSA(
              (qkv): Linear(in_features=96, out_features=288, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=96, out_features=96, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop): DropPath()
          )
          (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)

In [2]:
import os
os.listdir("datasets/EPFL/videos/basketball")

['match5-c2.avi', 'match5-c0.avi', 'match5-c1.avi', 'match5-c3.avi']

In [4]:
video = 'match5-c2.avi'
video2 = video.replace('.avi', '')
video2

'match5-c2'

In [1]:
import pandas as pd

COL_NAMES_EPFL = ('id', 'xmin', 'ymin', 'xmax', 'ymax', 'frame', 'lost', 'occluded', 'generated','label')
annotations_df = pd.read_csv("datasets/EPFL/annotations/basketball/match5-c0.txt", sep=' ', names=COL_NAMES_EPFL)
annotations_df = annotations_df[annotations_df.lost == 0]
annotations_df.head()

,id,xmin,ymin,xmax,ymax,frame,lost,occluded,generated,label
0,0,81,48,107,121,0,0,0,0,PERSON
1,0,80,48,106,121,1,0,0,1,PERSON
2,0,80,48,106,121,2,0,0,1,PERSON
3,0,80,48,106,121,3,0,0,1,PERSON
4,0,80,48,106,121,4,0,0,1,PERSON


In [2]:
annotations_df.label.unique()

array(['PERSON'], dtype=object)

In [3]:
annotations_df.frame.max(), annotations_df.size

(9367, 1042040)

In [4]:
annotations_df[(annotations_df.id == 1) & (annotations_df.lost == 0)]

,id,xmin,ymin,xmax,ymax,frame,lost,occluded,generated,label
9368,1,134,36,153,89,0,0,0,0,PERSON
9369,1,134,36,153,89,1,0,0,1,PERSON
9370,1,134,36,153,89,2,0,0,1,PERSON
9371,1,134,36,153,89,3,0,0,1,PERSON
9372,1,134,36,153,89,4,0,0,1,PERSON
...,...,...,...,...,...,...,...,...,...,...
18731,1,190,34,211,97,9363,0,0,1,PERSON
18732,1,189,34,210,97,9364,0,0,1,PERSON
18733,1,187,34,208,97,9365,0,0,1,PERSON
18734,1,186,34,207,97,9366,0,0,1,PERSON


In [5]:
annotations_df = annotations_df.sort_values(by="frame")
annotations_df.head()

,id,xmin,ymin,xmax,ymax,frame,lost,occluded,generated,label
18736,2,146,33,170,104,0,0,0,0,PERSON
37472,4,234,48,272,143,0,0,0,0,PERSON
28104,3,157,40,177,108,0,0,0,0,PERSON
9368,1,134,36,153,89,0,0,0,0,PERSON
56208,6,226,33,239,71,0,0,0,0,PERSON


## Exploring the connectivity of annotations

In [10]:
import torch
import numpy as np

frame_num = torch.from_numpy(annotations_df.frame.values)
max_frame_dist = 'max'
use_cuda=False
print(frame_num)

tensor([   0,    0,    0,  ..., 9367, 9367, 9367])


In [11]:
"""
Determines the valid connections among nodes (detections) according to their time distance. Valid connections
are those for which nodes are not in the same frame, and their time dist is not greater than max_frame_dist.
(Does the same as the function above at similar speed but does not require as much memory)
Args:
    frame_num: np.array with shape (num_nodes,), indicating the frame number of each node.
    max_frame_dist: maximum distance allowed among detections (in number of frames) (if 'max', it is ignored)
    use_cuda: bool indicates if operation must be performed in GPU
    return_undirected: bool, determines whether both (i,j) and (j, i) is returned for each edge

Returns:
    torch.Tensor with shape (2, num_edges) corresponding to the valid edges

"""
assert isinstance(max_frame_dist, (int, np.uint)) or max_frame_dist == 'max'

device = torch.device("cuda" if torch.cuda.is_available() and use_cuda else "cpu")
frame_num = frame_num.to(device)
assert (torch.sort(frame_num)[0] == frame_num).all(), "Detections are NOT sorted by timestamp. Graph was not created properly!"

changepoints = torch.where(frame_num[1:] != frame_num[:-1])[0] + 1
changepoints


tensor([     8,     16,     24,  ..., 104168, 104180, 104192])

In [12]:
changepoints = torch.cat((changepoints, torch.as_tensor([frame_num.shape[0]]).to(changepoints.device)))
all_det_ixs = torch.arange(frame_num.shape[0], device=frame_num.device)
print(changepoints, all_det_ixs)

tensor([     8,     16,     24,  ..., 104180, 104192, 104204]) tensor([     0,      1,      2,  ..., 104201, 104202, 104203])


In [ ]:
edge_ixs = []
for start_frame_ix, end_frame_ix in zip(changepoints[:-1], changepoints[1:]):
    curr_frame_ixs = all_det_ixs[start_frame_ix: end_frame_ix]
    curr_frame_num = frame_num[curr_frame_ixs[0]]

    # Update past frames
    if max_frame_dist != 'max':
        past_frames_ixs = torch.where(torch.abs(frame_num[:start_frame_ix] - curr_frame_num) <= max_frame_dist)[0]

    else:
        past_frames_ixs = all_det_ixs[:start_frame_ix]

    edge_ixs.append(torch.cartesian_prod(past_frames_ixs, curr_frame_ixs))

In [ ]:
edge_ixs = torch.cat(edge_ixs).T